In [1]:
import helpers as hlpr
import utility_functions as uf
import lottery as lt
import agent
from functools import partial

In [2]:
def print_HL_lottery(lottery):
    p1 = lottery[0]['prob']
    o1 = lottery[0]['out']
    p2 = lottery[1]['prob']
    o2 = lottery[1]['out']
    print(f"[{o1}]({p1}) : [{o2}]({p2})", end = "  ")   


def print_HL_lottery_pair(lottery_pair):
    first_lottery = lottery_pair[0]
    second_lottery = lottery_pair[1]
    print_HL_lottery(first_lottery)  
    print_HL_lottery(second_lottery)
    print()
    
def print_HL_lottery_pair_and_choice(lottery_pair, choice):
    first_lottery = lottery_pair[0]
    second_lottery = lottery_pair[1]
    print_HL_lottery(first_lottery)  
    if choice == 0:
        print("--A", end = " ")
        print_HL_lottery(second_lottery) 
        print()
    else:
        print("   ", end = " ")
        print_HL_lottery(second_lottery)
        print("--B")

        
def build_holt_laury_lotteries():
    """Returns list of Holt-Laury lottery pairs.
    
        returns:
            holt_laury_lotteries, list of lists
                inner list is a pair of lotteries for choice
    
    """
    #  Can we take advantage of the structure of lotteries in the Holt-Laury paper
    pay_a_high = 200*0.25974025974026
    pay_a_low = 160*0.25974025974026
    pay_b_high = 100
    pay_b_low =  10*0.25974025974026
    # Now we can build holt_laury_lotteries
    holt_laury_lotteries = []
    p = 0.0
    for k in range(9):
        p = round(p + 0.10, 2)
        lottery_a = [{'prob': p, 'out': pay_a_high}, {'prob': round(1.0-p, 2), 'out': pay_a_low}]
        lottery_b = [{'prob': p, 'out': pay_b_high}, {'prob': round(1.0-p, 2), 'out': pay_b_low}]
        holt_laury_lotteries.append([lottery_a, lottery_b])
    return holt_laury_lotteries

In [3]:
holt_laury_lotteries = build_holt_laury_lotteries()

print(holt_laury_lotteries[0])


[[{'prob': 0.1, 'out': 51.948051948052}, {'prob': 0.9, 'out': 41.5584415584416}], [{'prob': 0.1, 'out': 100}, {'prob': 0.9, 'out': 2.5974025974026}]]


In [4]:
def get_lottery_choices_og(holt_laury_lotteries, u):
    lottery_choices = []
    for lottery_pair in holt_laury_lotteries:
        k, eu = agent.lottery_choice(lottery_pair, u)
        lottery_choices.append(k)
    return lottery_choices

#def get_lottery_choices3(holt_laury_lotteries, u):
    lottery_choices = []
    for lottery_pair in holt_laury_lotteries:
        k, eu = lottery_choice_step(lottery_pair, u)
        lottery_choices.append(k)
    return lottery_choices

#def lottery_pair_eucalc(lottery,piecewise):
    lottery_pair_eu = []
    for piece in piecewise:
        x_min = piece[0].get("x_min")
        x_max = piece[0].get("x_max")
        if agent.expected_value(lottery) > x_min and agent.expected_value(lottery) < x_max:
            eu = 0.0
            for event in lottery:
                eu += (piece[0].get("slope") * event['out'] + piece[0].get("intercept")) * event['prob']
            lottery_pair_eu.append(eu)
    return lottery_pair_eu


#get_lottery_choices3(holt_laury_lotteries, x)

In [5]:
def certainty_equivalent(lottery, piecewise, min_x, max_x, precision = .01):
    """ Returns the certainty equivalent (ce) of a lottery.
        
            u(ce) = expected_utility(lottery, u)
            
            args:
                lottery, list of dictionaries.
                u, utility function defined over payoffs in dictionaries
            returns:
                ce, float, certainty equivalent
            TODO: only tested on linear_utility, needs to be tested with other utility functions
        """
    eu = 0.0
    for event in lottery:
        out = event['out']
        for piece in piecewise:
            if out > piece[0].get('x_min') and out < piece[0].get('x_max'):
                eu += (piece[0].get("slope") * event['out'] + piece[0].get("intercept")) * event['prob']
    #for event in lottery:
     #   eu += (piece[0].get("slope") * event['out'] + piece[0].get("intercept")) * event['prob']
    # now do a bisection search for x such that u(x) = eu
    lower_bound = min_x
    upper_bound = max_x
    num_trys = 0
    while(True):
        num_trys += 1
        u_upper = (piece[0].get("slope") * upper_bound) + piece[0].get("intercept")
        u_lower = (piece[0].get("slope") * lower_bound) + piece[0].get("intercept")
        try_this = .5*lower_bound + .5*upper_bound
        u_try = (piece[0].get("slope") * try_this) + piece[0].get("intercept")
        #print(f"lb = {lower_bound}, try = {try_this}, ub = {upper_bound}")
        #print(f" eu = {eu}, u({try_this}) = {u_try}")
        if hlpr.is_near_target(u_try, target = eu, precision = precision):
            return try_this, u_try, eu, num_trys 
        if u_try < eu:
            lower_bound = try_this
        else:
            upper_bound = try_this

In [6]:
def get_lottery_choices(holt_laury_lotteries, piecewise):
    lottery_choices = []
    for lottery_pair in holt_laury_lotteries:
        k = lottery_choice_step(lottery_pair, piecewise)[0]
        lottery_choices.append(k)
    return lottery_choices

def lottery_pair_eucalc(lottery_pair, piecewise):
    lottery_pair_eu = []
    for lottery in lottery_pair:
        eu = 0.0
        for event in lottery:
            out = event['out']
            for piece in piecewise:
                if out > piece[0].get('x_min') and out <= piece[0].get('x_max'):
                    eu += (piece[0].get("slope") * event['out'] + piece[0].get("intercept")) * event['prob']
        lottery_pair_eu.append(eu)
    return lottery_pair_eu

def lottery_choice_step(lottery_pair, piecewise):
    """ Choose the lottery with the highest expected utility
        from lottery_list using utility function u.
        
        returns:
            lottery_index, eu  expected utility of the lottery
    """
    lottery_index = None
    list_of_expected_u = []
    lottery_pair_utility = lottery_pair_eucalc(lottery_pair,piecewise)
    list_of_expected_u.append(lottery_pair_utility)
    choice = max(lottery_pair_utility)
    lottery_index = list_of_expected_u[0].index(choice)
    return lottery_index, choice

In [10]:
import risk

y = [   [     {       "slope": 0.48,       "intercept": 0,       "x_min": 0,       "x_max": 6.25     }   ],   [     {       "slope": 0.26666666666666666,       "intercept": 1.3333333333333333,       "x_min": 6.25,       "x_max": 25     }   ],   [     {       "slope": 0.16,       "intercept": 4,       "x_min": 25,       "x_max": 56.25     }   ],   [     {       "slope": 0.11428571428571428,       "intercept": 6.571428571428572,       "x_min": 56.25,       "x_max": 100     }   ] ]
result = []
util = y
hl_lots = build_holt_laury_lotteries()
choices = get_lottery_choices(hl_lots, y)
for lottery_pair, choice in zip(holt_laury_lotteries, choices):
    result.append(print_HL_lottery_pair_and_choice(lottery_pair, choice))

[51.948051948052](0.1) : [41.5584415584416](0.9)  --A [100](0.1) : [2.5974025974026](0.9)  
[51.948051948052](0.2) : [41.5584415584416](0.8)  --A [100](0.2) : [2.5974025974026](0.8)  
[51.948051948052](0.3) : [41.5584415584416](0.7)  --A [100](0.3) : [2.5974025974026](0.7)  
[51.948051948052](0.4) : [41.5584415584416](0.6)  --A [100](0.4) : [2.5974025974026](0.6)  
[51.948051948052](0.5) : [41.5584415584416](0.5)  --A [100](0.5) : [2.5974025974026](0.5)  
[51.948051948052](0.6) : [41.5584415584416](0.4)  --A [100](0.6) : [2.5974025974026](0.4)  
[51.948051948052](0.7) : [41.5584415584416](0.3)      [100](0.7) : [2.5974025974026](0.3)  --B
[51.948051948052](0.8) : [41.5584415584416](0.2)      [100](0.8) : [2.5974025974026](0.2)  --B
[51.948051948052](0.9) : [41.5584415584416](0.1)      [100](0.9) : [2.5974025974026](0.1)  --B
[None, None, None, None, None, None, None, None, None]


In [8]:
util = uf.crra_utility
choices = get_lottery_choices_og(holt_laury_lotteries, util)
for lottery_pair, choice in zip(holt_laury_lotteries, choices):
    print_HL_lottery_pair_and_choice(lottery_pair, choice)

[51.948051948052](0.1) : [41.5584415584416](0.9)  --A [100](0.1) : [2.5974025974026](0.9)  
[51.948051948052](0.2) : [41.5584415584416](0.8)  --A [100](0.2) : [2.5974025974026](0.8)  
[51.948051948052](0.3) : [41.5584415584416](0.7)  --A [100](0.3) : [2.5974025974026](0.7)  
[51.948051948052](0.4) : [41.5584415584416](0.6)  --A [100](0.4) : [2.5974025974026](0.6)  
[51.948051948052](0.5) : [41.5584415584416](0.5)  --A [100](0.5) : [2.5974025974026](0.5)  
[51.948051948052](0.6) : [41.5584415584416](0.4)  --A [100](0.6) : [2.5974025974026](0.4)  
[51.948051948052](0.7) : [41.5584415584416](0.3)      [100](0.7) : [2.5974025974026](0.3)  --B
[51.948051948052](0.8) : [41.5584415584416](0.2)      [100](0.8) : [2.5974025974026](0.2)  --B
[51.948051948052](0.9) : [41.5584415584416](0.1)      [100](0.9) : [2.5974025974026](0.1)  --B
